In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv(r'C:\\Users\\0819\\Desktop\\OCTOMBER\\NEW DEVELOPMENT\\Bitcoin time series\\CODE\\BACKEND\\bitcoin_2017_to_2023.csv')

In [8]:
! pip install prophet

Defaulting to user installation because normal site-packages is not writeable
     ---------------------                   7.2/13.3 MB 481.1 kB/s eta 0:00:13


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Program Files\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Program Files\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Program Files\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\Program Files\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Program Files\Python310\lib\ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Program Files\Python310\lib\ssl.py", line 1130, in read
    return self._sslobj.read(len, buffer)
TimeoutError: The read operation timed out

During handling of the above exception, another 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet  # Updated import
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

def prepare_data_for_prophet(stock_data):
    if not isinstance(stock_data, pd.Series):
        raise ValueError("stock_data must be a Pandas Series")
    
    # Convert index to datetime if necessary
    if not pd.api.types.is_datetime64_any_dtype(stock_data.index):
        stock_data.index = pd.to_datetime(stock_data.index)

    # Create a DataFrame with the required columns
    df = pd.DataFrame({'ds': stock_data.index, 'y': stock_data.values})
    return df

def process_stock_data_prophet(close_prices):
    print("Processing close prices for Prophet model...")

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler()
    close_prices_scaled = scaler.fit_transform(close_prices.values.reshape(-1, 1))

    # Prepare the data
    close_prices_scaled = pd.Series(close_prices_scaled.flatten(), index=close_prices.index)
    df = prepare_data_for_prophet(close_prices_scaled)

    # Initialize the Prophet model
    model = Prophet()

    # Fit the model
    model.fit(df)

    # Create a DataFrame for future predictions (e.g., predict for 6 months = 126 days)
    future = model.make_future_dataframe(periods=126)

    # Forecast
    forecast = model.predict(future)

    # Plot the results
    plt.figure(figsize=(14, 7))
    model.plot(forecast)
    plt.title('Close Price Prediction with Prophet')
    plt.xlabel('Date')
    plt.ylabel('Stock Price (scaled)')
    plt.show()

    # Evaluate the model on the last 126 days (the test set)
    test_data = df[-126:]  # Last 6 months of actual data
    forecasted_data = forecast[-126:]  # Last 6 months of predicted data

    # Reverse the scaling to compare actual and predicted values
    test_data_original = scaler.inverse_transform(test_data['y'].values.reshape(-1, 1)).flatten()
    forecasted_data_original = scaler.inverse_transform(forecasted_data['yhat'].values.reshape(-1, 1)).flatten()

    mse = mean_squared_error(test_data_original, forecasted_data_original)
    mae = mean_absolute_error(test_data_original, forecasted_data_original)
    rmse = np.sqrt(mse)
    r2 = r2_score(test_data_original, forecasted_data_original)

    print(f"""
    Evaluation Metrics for close prices:
    Mean Squared Error (MSE): {mse}
    Mean Absolute Error (MAE): {mae}
    Root Mean Squared Error (RMSE): {rmse}
    R² Score: {r2}
    """)

# Example usage:
# Assuming you have a DataFrame `df` with a 'close' column containing stock close prices
# process_stock_data_prophet(df['close'])
